# Import Libraries

In [1]:
from swarmintelligence import GreaterCaneRatAlgorithm
from swarmintelligence import Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))
rgb_misc_dataset = dict(itertools.islice(rgb_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
# optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
# optimal_params

In [5]:
optimal_params = {
    'ratSize':30,
    'rho': 0.5,
    'maxIteration': 10
}

# Evaluation

In [6]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
# n_runs = 30
n_runs = 1

In [7]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = GreaterCaneRatAlgorithm(
                        k=threshold,
                        ratSize=optimal_params['ratSize'],
                        rho=optimal_params['rho'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = GreaterCaneRatAlgorithm(
                        k=threshold,
                        ratSize=optimal_params['ratSize'],
                        rho=optimal_params['rho'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = GreaterCaneRatAlgorithm(
                        k=threshold,
                        ratSize=optimal_params['ratSize'],
                        rho=optimal_params['rho'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
552.3767737248421  |  2212.1072698116755  |  1349.4037919772904
Mean Fitness = 1371.2959451712693
	 AirplaneF16.tiff | Threshold =  3
956.0759462628585  |  2375.0679860818027  |  1783.407322096231
Mean Fitness = 1704.850418146964
	 AirplaneF16.tiff | Threshold =  4
624.6724573973736  |  2375.213786174715  |  1619.2941217868736
Mean Fitness = 1539.7267884529874
	 AirplaneF16.tiff | Threshold =  5
938.6147617031652  |  2522.5668501548766  |  1840.0638589078483
Mean Fitness = 1767.08182358863
	 Lena.png | Threshold =  2
914.3371837670137  |  2143.8455735352004  |  1872.1069398291056
Mean Fitness = 1643.4298990437735
	 Lena.png | Threshold =  3
780.28052012161  |  2326.2038994466056  |  1879.1101246398828
Mean Fitness = 1661.8648480693662
	 Lena.png | Threshold =  4
1044.7000600990143  |  2537.7609065522174  |  2091.874853064356
Mean Fitness = 1891.4452732385291
	 Lena.png | Threshold =  5
993.7130641974533  |  2387.9487260470178  |  2158.722824361196
Me

## Visualize Results using DataFrame

In [8]:
gwo_evaluation_results_df = pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[128, 180]","[93, 171]","[126, 193]",552.376774,2212.107270,1349.403792,...,"[[126, 255, 255, 255, 255, 255, 255, 255, 255,...",[552.3767737248421],[2212.1072698116755],[1349.4037919772904],[0.24305462837219238],[2239.7275988260903],[47.32576041466307],[14.628851592839759],[0.7663456509400755],[0.9409231039301321]
1,AirplaneF16.tiff,3,otsu,max,"[118, 153, 204]","[61, 133, 174]","[135, 177, 202]",956.075946,2375.067986,1783.407322,...,"[[135, 255, 202, 255, 255, 202, 202, 202, 202,...",[956.0759462628585],[2375.0679860818027],[1783.407322096231],[0.20947575569152832],[1511.1634483337402],[38.87368580844554],[16.337689204399325],[0.7287301776854423],[0.9589151840498542]
2,AirplaneF16.tiff,4,otsu,max,"[76, 103, 159, 198]","[59, 92, 127, 171]","[50, 102, 178, 202]",624.672457,2375.213786,1619.294122,...,"[[102, 255, 202, 255, 255, 202, 202, 202, 202,...",[624.6724573973736],[2375.213786174715],[1619.2941217868736],[0.20730304718017578],[1555.060036977132],[39.434249542461586],[16.213332001580756],[0.7330803070994096],[0.9625548114309893]
3,AirplaneF16.tiff,5,otsu,max,"[73, 108, 150, 177, 193]","[18, 99, 122, 169, 197]","[28, 100, 130, 159, 192]",938.614762,2522.566850,1840.063859,...,"[[100, 255, 255, 255, 255, 255, 255, 255, 255,...",[938.6147617031652],[2522.5668501548766],[1840.0638589078483],[0.2180314064025879],[1570.973056793213],[39.63550247938347],[16.169116241960936],[0.7682006673465889],[0.9617971926381185]
4,Lena.png,2,otsu,max,"[132, 186]","[90, 165]","[96, 140]",914.337184,2143.845574,1872.106940,...,"[[140, 140, 140, 140, 140, 140, 140, 140, 140,...",[914.3371837670137],[2143.8455735352004],[1872.1069398291056],[0.17551159858703613],[1915.3679733276367],[43.76491715207098],[15.308281395169168],[0.6366781054641312],[0.8894833972739015]
5,Lena.png,3,otsu,max,"[135, 176, 213]","[55, 119, 197]","[8, 91, 138]",780.280520,2326.203899,1879.110125,...,"[[138, 138, 138, 138, 138, 138, 138, 138, 138,...",[780.28052012161],[2326.2038994466056],[1879.1101246398828],[0.18269610404968262],[1497.4227778116863],[38.696547362932606],[16.377359258232605],[0.6533833201335748],[0.9110423239484263]
6,Lena.png,4,otsu,max,"[118, 159, 204, 226]","[58, 103, 148, 191]","[77, 97, 133, 166]",1044.700060,2537.760907,2091.874853,...,"[[133, 133, 166, 133, 133, 133, 133, 133, 133,...",[1044.7000600990143],[2537.7609065522174],[2091.874853064356],[0.18724870681762695],[674.8290252685547],[25.977471494904094],[19.83886606982128],[0.7344105322872521],[0.9452594357892088]
7,Lena.png,5,otsu,max,"[61, 108, 143, 192, 223]","[40, 59, 95, 121, 185]","[42, 83, 94, 107, 149]",993.713064,2387.948726,2158.722824,...,"[[149, 149, 149, 149, 149, 149, 149, 149, 149,...",[993.7130641974533],[2387.9487260470178],[2158.722824361196],[0.20247673988342285],[918.4067064921061],[30.305225729106624],[18.500453144487242],[0.7336837907278181],[0.9521839470760168]


# Save results

In [9]:
# pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_otsu_RGB_df.pkl', 'wb'))
# pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_otsu_RGB_dict.pkl', 'wb'))